In [1]:
#可視化とか
# !pip install -qq timm
# !pip install -qq albumentations==1.0.3
# !pip install -qq grad-cam
# !pip install -qq ttach
# !pip install GPUtil
# !pip install imbalanced-learn
# !pip install https://github.com/ufoym/imbalanced-dataset-sampler/archive/master.zip

## import

In [2]:
import gc
import sys
import math
import os
import pickle
import random
import re
import shutil
import time
import warnings
from collections import Counter, defaultdict
from contextlib import contextmanager

sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

import albumentations as A
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats as stats
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from albumentations import (
    Blur, Compose, Cutout, HorizontalFlip, IAAAdditiveGaussianNoise,
    ImageOnlyTransform, Normalize, OneOf, RandomBrightness, RandomBrightnessContrast,
    RandomContrast, RandomCrop, RandomResizedCrop, Resize, Rotate, ShiftScaleRotate, Transpose, VerticalFlip,
    HueSaturationValue, CoarseDropout, CenterCrop
)
from albumentations.pytorch import ToTensorV2
from functools import partial
from pathlib import Path
from PIL import Image
from scipy.special import softmax
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.preprocessing import LabelEncoder
from torch.nn.parameter import Parameter
from torch.nn.utils.rnn import pack_padded_sequence, pad_sequence
from torch.optim import SGD, Adam, AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts, ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
# from tqdm.auto import tqdm
import tqdm.notebook as tqdm

from IPython.display import Image, display_png
from sklearn.metrics import f1_score

warnings.filterwarnings('ignore')

In [3]:
# # いろいろ開放する
# import torch
# from GPUtil import showUtilization as gpu_usage
# from numba import cuda

# # setting the kernel accelerator to None and then back to GPU
# def free_gpu_cache():
#     print("Initial GPU Usage")
#     gpu_usage()                             

#     torch.cuda.empty_cache()

#     cuda.select_device(0)
#     cuda.close()
#     cuda.select_device(0)

#     print("GPU Usage after emptying the cache")
#     gpu_usage()
# free_gpu_cache()

## config

In [4]:

class CFG:
    debug = False
    seed = 460
    num_workers = 2
    batch_size = 64#256とか。画像枚数
    epochs = 100#あとで試す
    n_fold = 5
    size = 256
#     size = 64
    model_lr = 1e-5#lrとsizeもあとでチューニングする
    T_max = 10
    min_lr = 1e-6
    weight_decay = 1e-6
    gradient_accumulation_steps = 1
    max_grad_norm = 1000
    print_freq = 1000
    class_num = 10
    FOLDS = [0]
    device = "cuda" if torch.cuda.is_available() else "cpu"
#     MODEL_NAME = "resnet50"
    MODEL_NAME = 'tf_efficientnet_b0_ns'
    Version = "V1"

## util

In [5]:
def init_logger(log_file='train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger


def seed_torch(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


LOGGER = init_logger()
seed_torch(seed=CFG.seed)

In [6]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [7]:
def make_fold(df, stratified=True):
    """Add folds column."""
    df["folds"] = -1
    if stratified:
        kf = StratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=0)
        for f, (t_, v_) in enumerate(kf.split(X=df, y=df.classname.values)):
            df.loc[v_, 'folds'] = f
        df["folds"] = df["folds"].astype(int)

    else:
        kf = KFold(n_splits=CFG.n_fold, shuffle=True, random_state=0)
        for f, (t_, v_) in enumerate(kf.split(df)):
            df.loc[v_, 'folds'] = f
    return df


# def get_train_file_path(image_id, classname):
def get_train_file_path(train):
    """Get test img path."""
    return "../input/state-farm-distracted-driver-detection/imgs/train/{}/{}".format(
        train.classname, train.img
    )

## dataloader

In [8]:
class TrainDataset(Dataset):

    def __init__(self, df, transform=None):
        self.df = df
        self.labels = df['classnameInt'].values
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
#         print(self.df.loc[idx]["file_paths"])
        image = cv2.imread(self.df.loc[idx]["file_paths"])
#         print(image)
        
#         image = self.df.loc[idx, "pixel0": "pixel783"].values.astype(np.uint8).reshape(28, 28)
#         image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        label = self.labels[idx]
        return image, label

In [9]:
IMG_MEAN = [0.4452, 0.4457, 0.4464]
IMG_STD = [0.2592, 0.2596, 0.2600]

def get_transforms(*, data, size):

    if data == 'train':
        return A.Compose([
            A.Resize(size,size),
            A.Normalize(
                mean=IMG_MEAN,
                std=IMG_STD,
            ),
            A.HorizontalFlip(),
            A.RandomBrightness(),
            ToTensorV2(), # Tensor型に変換
        ])

    elif data == 'valid':
        return A.Compose([
            A.Resize(size,size),
            A.Normalize(
                mean=IMG_MEAN,
                std=IMG_STD,
            ),
            ToTensorV2(),
        ])

## model

In [10]:
# use timm model
class BaseModel(nn.Module):
    def __init__(
        self, model_name=CFG.MODEL_NAME, n_class=CFG.class_num, pretrained=False
    ):
        super().__init__()
        #backboneとして使ってheadだけつけかえてもいい。ReLuとBatchNormとか入れる？
        #そうした場合forwardに使う順番をかく
        self.backbone = timm.create_model(model_name, pretrained=pretrained, num_classes = 0)
        
        self.in_features = self.backbone.num_features
        self.head = nn.Sequential(
                    nn.Linear(self.in_features, 1024),
                    nn.ReLU(),
                    nn.Dropout(),
                    nn.Linear(1024, 512),
                    nn.Dropout(),
                    nn.Linear(512, n_class)
                    )

    def forward(self, x):
        x = self.backbone(x)
        x = self.head(x)
        output = x
        #         output = self.model(x)
        return output

## Loss

In [11]:
class FocalLoss(nn.Module):#今は使っていない

    def __init__(self, gamma=0, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.eps = eps
        self.ce = torch.nn.CrossEntropyLoss(reduction="none")

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()

In [12]:
def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    #avg_score = AverageMeter()

    model.train()
    start = end = time.time()
    global_step = 0

    for step, (images, labels) in tqdm.tqdm(enumerate(train_loader)):#tqdm使う？
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(CFG.device)
#         torch.from_numpy(np.asarray(labels))
        labels = torch.from_numpy(np.array(labels))
        labels = labels.to(CFG.device)
        batch_size = labels.size(0)
        y_preds = model(images)
        loss = criterion(y_preds, labels)

        losses.update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        else:
            loss.backward()
            
        del loss
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(
                   epoch+1, step, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(train_loader)),
                   grad_norm=grad_norm,
                   ))

        del y_preds, images, labels, batch_size
    del batch_time, data_time, scores, model
    gc.collect()
    torch.cuda.empty_cache()
    
    return losses.avg

In [13]:
def valid_fn(valid_loader, model, criteron):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to evaluation mode
    model.eval()
    preds = []
    confs = []
    start = end = time.time()
    for step, (images, labels) in tqdm.tqdm(enumerate(valid_loader)):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(CFG.device)
        labels = labels.to(CFG.device)
        batch_size = labels.size(0)
        # compute loss
        with torch.no_grad():
        #with torch.inference_mode():
            y_preds = model(images)
            loss = criteron(y_preds, labels)
            losses.update(loss.item(), batch_size)
            # record accuracy
            preds.append(y_preds.softmax(1).to("cpu").numpy().argmax(1))
            
            if CFG.gradient_accumulation_steps > 1:
                loss = loss / CFG.gradient_accumulation_steps
            
            del loss
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
            if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
                print('EVAL: [{0}/{1}] '
                    'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                    'Elapsed {remain:s} '
                    'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                    .format(
                    step, len(valid_loader), batch_time=batch_time,
                    data_time=data_time, loss=losses,
                    remain=timeSince(start, float(step+1)/len(valid_loader)),
                    ))
        del y_preds, images, labels, batch_size
    del batch_time, data_time, scores, model
    gc.collect()
    torch.cuda.empty_cache()

    preds = np.concatenate(preds)

    return losses.avg, preds
            

## データ読み込み

In [14]:
# データ読み込み
IMG_EXTENSIONS = [ '.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP', ]

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

def find_classes(dir):
    classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx

# データ読みこみ
# train_dir = '../input/state-farm-distracted-driver-detection/imgs/train'
# test_dir = '../input/state-farm-distracted-driver-detection/imgs/test'

# classes = ['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9']
# train = make_fold(pd.read_csv("../input/animal-imbalance-classification/train.csv"))
train = make_fold(pd.read_csv("../input/state-farm-distracted-driver-detection/driver_imgs_list.csv"))
# train["file_paths"] = train.img.map(get_train_file_path)
for idx, rows in train.iterrows():
    train.at[idx, 'file_paths'] = get_train_file_path(train.iloc[idx])
classes_map = {'c0':0,'c1':1,'c2':2,'c3':3,'c4':4,'c5':5,'c6':6,'c7':7,'c8':8,'c9':9}
train['classnameInt'] = train.classname.map(lambda x: classes_map[x])
    

## 学習

In [15]:
def train_loop(train, fold, class_num=None):
    #fold複数試してもいいかも->一番下でやってた
    train_index = train[train['folds'] != fold].index
    valid_index = train[train['folds'] == fold].index

    train_dataset = TrainDataset(train.loc[train_index].reset_index(drop=True), 
                                 transform=get_transforms(data='train', size=CFG.size))
    valid_dataset = TrainDataset(train.loc[valid_index].reset_index(drop=True), 
                                 transform=get_transforms(data='valid', size=CFG.size))
# 
#     balanced_batch_sampler = BalancedBatchSampler(train_dataset, 10, CFG.batch_size)
    
    train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
#     train_loader = DataLoader(train_dataset,   
#                               num_workers=CFG.num_workers, pin_memory=True, batch_sampler = balanced_batch_sampler)
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=False, 
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    
    
    model = BaseModel(CFG.MODEL_NAME, CFG.class_num, pretrained=False).to(CFG.device)

    optimizer = Adam(model.parameters(), lr=CFG.model_lr, weight_decay=CFG.weight_decay, amsgrad=False)
    scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)

    criterion = nn.CrossEntropyLoss()
#     criterion = FocalLoss()

    best_score = 0.
    best_loss = np.inf
    
    for epoch in range(CFG.epochs):
        start_time = time.time()
        avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, scheduler)
        avg_val_loss, predict = valid_fn(valid_loader, model, criterion)
        valid_labels = train.loc[valid_index, "classnameInt"].values
        
        scheduler.step()
        #score = accuracy_score(valid_labels, predict)
        score = f1_score(valid_labels, predict, average="micro")
        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Accuracy: {score}')

        if score > best_score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save(model.state_dict(), "./" + f'{CFG.MODEL_NAME}_fold{fold}_best_{CFG.Version}.pth')
#             print(model.state_dict(), "./" + f'{CFG.MODEL_NAME}_fold{fold}_best_{CFG.Version}.pth')
            valid_folds_predict = predict
            
        del avg_loss, avg_val_loss, valid_labels, predict, score
        gc.collect()
        torch.cuda.empty_cache()

    return valid_folds_predict

In [16]:
# 学習
oof_df = pd.DataFrame()
preds = []
for fold in range(CFG.n_fold):
    if fold not in CFG.FOLDS:
        continue

    print(fold)
    if CFG.debug:
        train = train.iloc[:2000]
    pred = train_loop(train, fold, CFG.class_num)
    preds.append([pred])
    LOGGER.info(f"========== fold: {fold} result ==========")

0


0it [00:00, ?it/s]

Epoch: [1][0/280] Data 1.642 (1.642) Elapsed 0m 3s (remain 15m 34s) Loss: 2.2863(2.2863) 
Epoch: [1][279/280] Data 0.199 (0.301) Elapsed 3m 6s (remain 0m 0s) Loss: 2.3142(2.3036) 


0it [00:00, ?it/s]

EVAL: [0/71] Data 1.217 (1.217) Elapsed 0m 1s (remain 1m 32s) Loss: 2.2762(2.2762) 
EVAL: [70/71] Data 0.000 (0.459) Elapsed 0m 38s (remain 0m 0s) Loss: 2.0585(2.2993) 


...

0it [00:00, ?it/s]

Epoch: [100][0/280] Data 1.207 (1.207) Elapsed 0m 1s (remain 7m 39s) Loss: 0.8220(0.8220) 
Epoch: [100][279/280] Data 0.001 (0.174) Elapsed 2m 36s (remain 0m 0s) Loss: 0.8903(0.8591) 


0it [00:00, ?it/s]

EVAL: [0/71] Data 0.852 (0.852) Elapsed 0m 0s (remain 1m 8s) Loss: 2.4473(2.4473) 
EVAL: [70/71] Data 0.000 (0.312) Elapsed 0m 28s (remain 0m 0s) Loss: 0.0042(1.3719) 


Epoch 100 - avg_train_loss: 0.8591  avg_val_loss: 1.3719  time: 186s
Epoch 100 - Accuracy: 0.5748049052396879
========== fold: 0 result ==========


In [17]:
# class TrainDataset(Dataset):

#     def __init__(self, df, transform=None):#まとめてdfを渡す
#         self.df = df
#         self.labels = df['classname'].values
#         self.transform = transform

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         image = cv2.imread(self.df.loc[idx]["file_paths"])
# #         print(image)
        
# #         image = self.df.loc[idx, "pixel0": "pixel783"].values.astype(np.uint8).reshape(28, 28)
# #         image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

#         if self.transform:
#             augmented = self.transform(image=image)
#             image = augmented['image']

#         label = self.labels[idx]
#         return image, label
    
# train = make_fold(pd.read_csv("../input/state-farm-distracted-driver-detection/driver_imgs_list.csv"))
# for idx, rows in train.iterrows():
#     train.at[idx, 'file_paths'] = get_train_file_path(train.iloc[idx])
# train_index = train[train['folds'] != fold].index
# # print(train.loc[train_index].reset_index(drop=True))

# classes_map = {'c0':0,'c1':1,'c2':2,'c3':3,'c4':4,'c5':5,'c6':6,'c7':7,'c8':8,'c9':9}
# train['classnameInt'] = train.classname.map(lambda x: classes_map[x])
# train.head()

# train_dataset = TrainDataset(train.loc[train_index].reset_index(drop=True), 
#                                  transform=get_transforms(data='train', size=CFG.size))
# train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True,
#                               num_workers=CFG.num_workers, pin_memory=True, drop_last=True)

# for step, (images, labels) in tqdm.tqdm(enumerate(train_loader)):
# #     print(images)
#     print(labels)

In [18]:
# for idx, rows in train.iterrows():
#     train.at[idx, 'file_paths'] = get_train_file_path(train.iloc[idx])

In [19]:
# train = pd.read_csv("../input/state-farm-distracted-driver-detection/driver_imgs_list.csv")
# train["file_paths"] = train.img.map(get_train_file_path)
# train.head()